In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, NoSuchElementException
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import requests
import itertools
import time
import csv

In [2]:
def get_cat(full_url):

    # Selenium driver
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15'
    path = '/Users/emma/Documents/chromedriver'
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--window-size=1920,1080')
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--allow-running-insecure-content')
    options.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.get(full_url)

    # Find class for tags with URLs to loop through
    elems = driver.find_elements_by_tag_name('a')
    cat_list = [elem.get_attribute('href') for elem in elems]

    driver.quit()

    return cat_list

In [3]:
# Get all URLs and remove NoneType values
cat_list = get_cat('https://www.boobdesign.com/')
cat_list = [string for string in cat_list if string]

# Filter URLs & Remove duplicate URLs
substring = ['shop']
cat_list = list(set([string for string in cat_list if any(sub in string for sub in substring)]))

cat_dict = {}

In [4]:
def cat_itemize(cat_url):

    # Selenium driver
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15'
    path = '/Users/emma/Documents/chromedriver'
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument("--window-size=1920,1080")
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--allow-running-insecure-content')
    options.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Chrome(executable_path=path, options=options)

    # Name of the category
    HEADERS = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'}
    content = requests.get(cat_url, headers=HEADERS)
    soup = BeautifulSoup(content.text, 'html.parser')
    cat_name = soup.find('h1', {'class': 'up'}).get_text(strip=True)
    
    if cat_name not in cat_dict:
        driver.get(cat_url)

        # Assign values for container of all items, invdividual items, item URL
        all_items = soup.find('ul', {'class': 'productlist row'}).find_all('li', {'class':'small-puff columns small-6 medium-4'})
        cat_dict[cat_name] = ['https://www.boobdesign.com'+items.find('a', {'class':'fill'}).get('href') for items in all_items]
        
    return cat_dict

In [5]:
for i in range(len(cat_list)):
    cat_dict_load = cat_itemize(cat_list[i])

# Join dictionary values (url lists) into one list
boob_url_list = []

for i in cat_dict_load:
    l = cat_dict_load[i]
    for j in range(len(l)):
        boob_url_list.append(l[j])

# Remove duplicate urls
boob_url_list = list(set(boob_url_list))

# Remove uneccssary items 
exclude = ['gift-card', 'tgc']
boob_url_list = [string for string in boob_url_list if any(sub not in string for sub in exclude)]   

url_list = []

In [6]:
# Scraper function
def boob_scraper(boob_url_list, url_list):

    # Define empty lists to store results and log failed URLs
    failed = []
    results = []

    for boob_url in tqdm(boob_url_list):
        if boob_url not in url_list:
            # Empty dictionary to store output
            boob_results = {}

            # Beautiful soup driver
            HEADERS = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'}
            content = requests.get(boob_url, headers=HEADERS)
            soup = BeautifulSoup(content.text, 'html.parser')

            # Selenium driver
            user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'
            path = '/Users/emma/Documents/chromedriver'
            options = webdriver.ChromeOptions()
            options.add_argument('--headless')
            options.add_argument('--window-size=1920,1080')
            options.add_argument('--ignore-certificate-errors')
            options.add_argument('--allow-running-insecure-content')
            options.add_argument(f'user-agent={user_agent}')
            driver = webdriver.Chrome(executable_path=path, options=options)
            driver.get(boob_url)
            
            try:
                # Product name
                boob_name = soup.find('h1', {'itemprop': 'name'}).get_text(strip=True)
                boob_results['Name'] = boob_name

                # Product material
                for i in soup.find('div',{'class': 'description'}).find_all('li'):
                    if '%' in i.text.lower() and ':' in i.text: 
                        boob_material = i.text
                boob_results['Material'] = boob_material
                
                # Product color
                boob_color = []
                for i in soup.find('div',{'class': 'colors'}).find_all('img'):
                    boob_color.append(i.get('title'))
                boob_results['Color'] = boob_color

                # Product size
                boob_size = []
                for i in soup.find('div',{'class':'sizes'}).find_all('a',{'class':'button'}):
                    boob_size.append(i.get_text(strip=True))
                boob_results['Size'] = boob_size
                
                # Product price
                boob_price = soup.find('div', {'class': 'price'}).get_text(strip=True)
                boob_results['Price'] = boob_price

                # Product URL
                boob_results['URL'] = boob_url

                # Product image
                boob_image = []
                for i in soup.find('div', {'class': 'slider'}).find_all('div'):
                    boob_image.append(i.find('img').get('src'))
                boob_results['Image'] = boob_image
                
                # Product description
                boob_description = soup.find('div',{'class':'description'}).get_text(strip=True)
                boob_results['Description'] = boob_description

                # Add dictionary output to list
                results.append(boob_results)

            except (NoSuchElementException, AttributeError, TypeError):
                if NoSuchElementException:
                    failed.append(boob_url)
                elif AttributeError:
                    failed.append(boob_url)
                elif TypeError:
                    failed.append(boob_url)
                pass
    
            with open('boob_table.csv', mode='w') as csv_file:
                writer = csv.DictWriter(csv_file, fieldnames=['Name', 'Material', 'Color', 'Size', 'Price', 'URL', 'Image', 'Description'])
                writer.writeheader()
                writer.writerows(results)

            with open('failed_boob.txt', mode='w') as txt_file:
                for row in failed:
                    txt_file.write(str(row) + '\n')

boob_scraper(boob_url_list, url_list)

100%|██████████| 126/126 [12:49<00:00,  6.11s/it]


In [87]:
df = pd.read_csv('boob_table.csv')
df['Brand_name'] = 'Boob'
df = df[['Name', 'Material', 'Color', 'Size', 'Price', 'URL', 'Image', 'Brand_name', 'Description']]
df.head()

,Name,Material,Color,Size,Price,URL,Image,Brand_name,Description
0,B.Warmer sweatshirt,Sustainable materials: 92% organic cotton and ...,"['Light pink', 'Olive leaf', 'Steel blue']","['XS', 'S', 'M', 'L', 'XL']",109.00 USD,https://www.boobdesign.com/products/maternity-...,['https://1118730033.rsc.cdn77.org/storage/C63...,Boob,Our B.Warmer sweatshirt is as street-smart as ...
1,Air halterneck dress,Sustainable material: 100% organic cotton,"['Midnight blue', 'Port red']","['S', 'M', 'L']",159.00 USD,https://www.boobdesign.com/products/maternity-...,['https://1118730033.rsc.cdn77.org/storage/997...,Boob,"Light, airy and cut generously to ensure a gre..."
2,Debbie dress,Sustainable material: 95% lyocell and 5% elastane,['Steel blue'],"['S/M', 'L/XL']",56.70 USD,https://www.boobdesign.com/outlet/outlet-70/de...,['https://1118730033.rsc.cdn77.org/storage/0FF...,Boob,With its V-neck and chic dolman sleeves this l...
3,Signe sleeveless dress,Sustainable material: 94% organic cotton and 6...,"['Black', 'Pine green']","['S', 'M', 'L']",119.00 USD,https://www.boobdesign.com/products/maternity-...,['https://1118730033.rsc.cdn77.org/storage/52D...,Boob,A figure-hugging ribbed dress designed to adap...
4,Signe dress,Sustainable material: 94% organic cotton and 6...,"['Black', 'Hazelnut']","['S', 'M', 'L']",139.00 USD,https://www.boobdesign.com/products/maternity-...,['https://1118730033.rsc.cdn77.org/storage/7D2...,Boob,There are clothes you want to live in. Like Si...


In [88]:
def material(x):
    l = []
    for i in x.split(':')[-1].strip().split(';'):
        if 'and' in i.strip():
            [l.append(i.strip()) for i in i.strip().split('and')]
        else:
            l.append(i.strip())
    return l

def price(x):
    return '$' + x[:-3].strip()

df['Material'] = df['Material'].apply(lambda x: material(x))
df['Price'] = df['Price'].apply(lambda x: price(x))
df['Description'] = df['Description'].apply(lambda x: x.split('.'))

In [89]:
df

,Name,Material,Color,Size,Price,URL,Image,Brand_name,Description
0,B.Warmer sweatshirt,"[92% organic cotton, 8% elastane, the fleece l...","['Light pink', 'Olive leaf', 'Steel blue']","['XS', 'S', 'M', 'L', 'XL']",$109.00,https://www.boobdesign.com/products/maternity-...,['https://1118730033.rsc.cdn77.org/storage/C63...,Boob,"[Our B, Warmer sweatshirt is as street-smart a..."
1,Air halterneck dress,[100% organic cotton],"['Midnight blue', 'Port red']","['S', 'M', 'L']",$159.00,https://www.boobdesign.com/products/maternity-...,['https://1118730033.rsc.cdn77.org/storage/997...,Boob,"[Light, airy and cut generously to ensure a gr..."
2,Debbie dress,"[95% lyocell, 5% elastane]",['Steel blue'],"['S/M', 'L/XL']",$56.70,https://www.boobdesign.com/outlet/outlet-70/de...,['https://1118730033.rsc.cdn77.org/storage/0FF...,Boob,[With its V-neck and chic dolman sleeves this ...
3,Signe sleeveless dress,"[94% organic cotton, 6% elastane]","['Black', 'Pine green']","['S', 'M', 'L']",$119.00,https://www.boobdesign.com/products/maternity-...,['https://1118730033.rsc.cdn77.org/storage/52D...,Boob,[A figure-hugging ribbed dress designed to ada...
4,Signe dress,"[94% organic cotton, 6% elastane]","['Black', 'Hazelnut']","['S', 'M', 'L']",$139.00,https://www.boobdesign.com/products/maternity-...,['https://1118730033.rsc.cdn77.org/storage/7D2...,Boob,"[There are clothes you want to live in, Like ..."
...,...,...,...,...,...,...,...,...,...
120,Fast Food bra - Classic,"[91% recycled polyamide, 9% elastane]","['Black', 'Soft pink', 'White']","['S', 'M', 'L', 'XL']",$59.00,https://www.boobdesign.com/products/maternity-...,['https://1118730033.rsc.cdn77.org/storage/867...,Boob,[Flexibility is everything when it comes to nu...
121,The-shirt dress,[100% organic cotton],"['Black', 'Willow green']","['S', 'M', 'L']",$109.00,https://www.boobdesign.com/products/maternity-...,['https://1118730033.rsc.cdn77.org/storage/B42...,Boob,[Our best-selling The-shirt top in dress form!...
122,Classic long-sleeved top,"[92% organic cotton, 8% elastane]","['Black', 'Midnight blue', 'White']","['XS', 'S', 'M', 'L', 'XL', 'XXL']",$79.00,https://www.boobdesign.com/products/maternity-...,['https://1118730033.rsc.cdn77.org/storage/E1A...,Boob,[This award-winning classic is a favorite with...
123,Breton short-sleeved top,[100% organic cotton],['Midnight blue/Tofu'],"['S', 'M', 'L']",$109.00,https://www.boobdesign.com/products/maternity-...,['https://1118730033.rsc.cdn77.org/storage/AF4...,Boob,"[A timeless classic to fall in love with, An ..."


In [90]:
df.to_csv('boob_table.csv')